## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 79.2 MB/s 
     |████████████████████████████████| 7.6 MB 88.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 101.7 MB/s 
     |████████████████████████████████| 212 kB 97.9 MB/s 
     |████████████████████████████████| 127 kB 84.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels

In [2]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATSC
from InstructABSA.data_prep import ModelReadyData

In [4]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [5]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()
res_tr_df = model_data.extract_rowwise_aspect_polarity(res_tr_df, on='aspectTerms', by="term", min_val=1)
lap_tr_df = model_data.extract_rowwise_aspect_polarity(lap_tr_df, on='aspectTerms', by="term", min_val=1)
res_te_df = model_data.extract_rowwise_aspect_polarity(res_te_df, on='aspectTerms', by="term", min_val=1)
lap_te_df = model_data.extract_rowwise_aspect_polarity(lap_te_df, on='aspectTerms', by="term", min_val=1)

 # Get the input text into the required format
bos_instruction = 'Review: '
delim_instruction = ' Aspect: '
eos_insrtuction = ' Sentiment:'
res_tr_df = model_data.create_data_in_atsc_format(res_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)
lap_tr_df = model_data.create_data_in_atsc_format(lap_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)
res_te_df = model_data.create_data_in_atsc_format(res_te_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)
lap_te_df = model_data.create_data_in_atsc_format(lap_te_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)

## 10% Training Data

In [6]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.1)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', '__index_level_0__'],
        num_rows: 863
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 2360
    })
})

In [8]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 863
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 216
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.314146
2,No log,0.286975
3,No log,0.296614
4,No log,0.270092


***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/driv

In [9]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 863
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 2360
  Batch size = 16


In [10]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '10pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '10pc_' + id_te_filename), index = False)

Train accuracy:  90.38238702201622
Test accuracy:  83.38983050847457


## 20% Training Data

In [12]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.2)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
  

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "pre

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', '__index_level_0__'],
        num_rows: 1726
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 2360
    })
})

In [14]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1726
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 432
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a 

Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.273942
2,No log,0.233755
3,No log,0.222326
4,No log,0.215531


***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/driv

In [15]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1726
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 2360
  Batch size = 16


In [17]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '20pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '20pc_' + id_te_filename), index = False)

Train accuracy:  92.69988412514485
Test accuracy:  85.76271186440678


## 50% Training Data

In [18]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.5)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
  

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "pre

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints


  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [20]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Trainer device: cuda:0


The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 4314
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1080
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.336964
2,0.953300,0.204791
3,0.953300,0.216446
4,0.151700,0.219179


***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/driv

In [21]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 4314
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-combined100_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 2360
  Batch size = 16


In [22]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '50pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '50pc_' + id_te_filename), index = False)

Train accuracy:  94.13537320352341
Test accuracy:  86.44067796610169
